In [ ]:
import sys
if 'google.colab' in str(get_ipython()):
  ! {sys.executable} -m pip install pytorch-lifestream
  ! pip install "torch<2"
  ! pip install torchvision==0.13.1

In [3]:
import os

if not os.path.exists('data/train.csv'):
  ! mkdir -p data
  ! wget https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/31erlljbW6Px9A -O data-science-bowl-2019.zip
  ! unzip -j data-science-bowl-2019.zip -d data
  ! mv data-science-bowl-2019.zip data/

--2024-05-07 09:17:41--  https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/31erlljbW6Px9A
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/c91216963c8d2a9a57b33d06b5de1deef31686c4d9d0f2cd56a8fd9c4e752ffc/663a2853/uL5QNWQw5yagmnYON70lZ6Z0tvI8cp2_OwrICh06gTpgEGVvh-xhxaiXJLc59hjIXVPF_DeMVCQZOCeaFO00oA%3D%3D?uid=0&filename=data-science-bowl-2019.zip&disposition=attachment&hash=Tlvpd2GJrqcVeirWz2GOlKqlSJpjhxNCqS1t1aV5dBrIz6FlUvfKjsTrXByCF6GRq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=509284498&fsize=448890651&hid=1aa5f98e72e3a8949e278f4915c698ba&media_type=compressed&tknv=v2 [following]
--2024-05-07 09:17:42--  https://downloader.disk.yandex.ru/disk/c91216963c8d2a9a57b33d06b5de1deef31686c4d9d0f2cd56a8fd9c4e752ffc/663a2853/uL5QNWQw5yagmnYON70lZ6

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
%load_ext autoreload
%autoreload 2

import torch
import pytorch_lightning as pl

In [5]:
import pandas as pd
data = pd.read_csv('data/train.csv')

In [6]:
data.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,event_type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [7]:
train_labels = pd.read_csv('data/train_labels.csv')
train_labels

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3
...,...,...,...,...,...,...,...
17685,c996482b11d149dd,ffc90c32,Bird Measurer (Assessment),1,0,1.000000,3
17686,b05a02b52d5c1f4c,ffd2871d,Cauldron Filler (Assessment),1,0,1.000000,3
17687,5448d652309a6324,ffeb0b1b,Cauldron Filler (Assessment),1,2,0.333333,1
17688,a6885ab824fbc32c,ffeb0b1b,Mushroom Sorter (Assessment),0,1,0.000000,0


In [8]:
data = data.drop(columns = ['event_data', 'installation_id'])

In [9]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
data['world'] = label_enc.fit_transform(data['world'])
data['event_type'] = label_enc.fit_transform(data['event_type'])
data['event_id'] = label_enc.fit_transform(data['event_id'])
data['game_session'] = label_enc.fit_transform(data['game_session'])
data['title'] = label_enc.fit_transform(data['title'])



data.head()

,event_id,game_session,timestamp,event_count,event_code,game_time,title,event_type,world
0,53,82641,2019-09-06T17:53:46.937Z,1,2000,0,43,2,2
1,53,28340,2019-09-06T17:54:17.519Z,1,2000,0,28,2,1
2,189,9756,2019-09-06T17:54:56.302Z,1,2000,0,35,0,1
3,283,9756,2019-09-06T17:54:56.387Z,2,3010,53,35,0,1
4,40,9756,2019-09-06T17:55:03.253Z,3,3110,6972,35,0,1


In [10]:
from datetime import datetime
data['timestamp'] = data['timestamp'].apply(
    lambda x: datetime.fromisoformat(x.replace("Z", "+00:00")).timestamp()
)

In [11]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='game_session',
    col_event_time='timestamp',
    event_time_transformation='none',
    cols_category=['event_id', 'event_code', 'event_type', 'world', 'title'],
    cols_numerical=['event_count', 'game_time'],
    return_records=True,
)

In [12]:
%%time

dataset = preprocessor.fit_transform(data)

CPU times: user 4min 20s, sys: 16.3 s, total: 4min 36s
Wall time: 4min 34s


In [13]:
import pickle

with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [14]:
dataset = sorted(dataset, key=lambda x: x['game_session'])

In [15]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.1, random_state=42)

len(train), len(test)

(272987, 30332)

In [16]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'event_time': 'identity'},
    embeddings={
        'timestamp': {'in': 800, 'out': 16},
        'event_code': {'in': 50, 'out': 8},
        'event_type': {'in': 4, 'out': 4},
        'world': {'in': 4, 'out': 4},
        'event_count': {'in': 3500, 'out': 16},
        'game_time': {'in': 834050, 'out': 16},
        'event_id': {'in': 385, 'out': 16}

    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)

In [17]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

train_dl = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=train,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=25,
            cnt_max=200,
        ),
    ),
    train_num_workers=2,
    train_batch_size=256,
)

In [18]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

In [ ]:
torch.save(seq_encoder.state_dict(), "coles-emb.pt")

In [ ]:
from ptls.data_load.datasets import inference_data_loader

train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = torch.vstack(trainer.predict(model, test_dl))

# test_embeds

In [ ]:
df_target = train_labels

df_target = df_target.set_index('game_session')
df_target.rename(columns={"accuracy_group": "target"}, inplace=True)
df_target


In [ ]:
df_target = df_target[['game_session', 'target']]

In [ ]:
train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['game_session'] = [x['game_session'] for x in train]
train_df = train_df.merge(df_target, how='inner', on='game_session')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['game_session'] = [x['game_session'] for x in test]
test_df = test_df.merge(df_target, how='inner', on='game_session')
print(train_df.shape, test_df.shape)


In [21]:
from sklearn.ensemble import RandomForestClassifier

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.623823
